## Mounting Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd 'drive/My Drive/M5'

/content/drive/My Drive/M5


## Import statements

In [0]:
import pandas as pd
from fbprophet import Prophet

In [0]:
file_name = "sales_1"

In [0]:
cal_df = pd.read_csv("calendar.csv")

In [6]:
cal_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1969 entries, 0 to 1968
Data columns (total 14 columns):
date            1969 non-null object
wm_yr_wk        1969 non-null int64
weekday         1969 non-null object
wday            1969 non-null int64
month           1969 non-null int64
year            1969 non-null int64
d               1969 non-null object
event_name_1    162 non-null object
event_type_1    162 non-null object
event_name_2    5 non-null object
event_type_2    5 non-null object
snap_CA         1969 non-null int64
snap_TX         1969 non-null int64
snap_WI         1969 non-null int64
dtypes: int64(7), object(7)
memory usage: 215.5+ KB


In [7]:
cal_df.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [0]:
date_map_df = cal_df[["date", "d", "snap_CA", "snap_TX", "snap_WI", "wm_yr_wk"]]

In [0]:
date_map_df = date_map_df.rename(columns={"date": "ds"})

In [0]:
future_dates = list(date_map_df[1913:1941].ds)

In [0]:
sell_prices_df = pd.read_csv("sell_prices.csv")

In [0]:
sell_prices_df["id"] = sell_prices_df["item_id"] + "_" + sell_prices_df["store_id"] + "_validation"

In [0]:
sell_prices_df = sell_prices_df[["id", "wm_yr_wk", "sell_price"]]

In [15]:
sell_prices_df.head()

,id,wm_yr_wk,sell_price
0,HOBBIES_1_001_CA_1_validation,11325,9.58
1,HOBBIES_1_001_CA_1_validation,11326,9.58
2,HOBBIES_1_001_CA_1_validation,11327,8.26
3,HOBBIES_1_001_CA_1_validation,11328,8.26
4,HOBBIES_1_001_CA_1_validation,11329,8.26


In [0]:
sales_train_df = pd.read_csv(file_name + ".csv", index_col=0)

In [17]:
sales_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1016 entries, 1016 to 2031
Columns: 1919 entries, id to d_1913
dtypes: int64(1913), object(6)
memory usage: 14.9+ MB


In [18]:
sales_train_df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,...,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
1016,HOUSEHOLD_1_461_CA_1_validation,HOUSEHOLD_1_461,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,1,0,2,0,0,1,0,3,1,1,1,0,2,1,3,1,5,1,2,1,0,0,2,0,0,1,1,1,1,3,10,1,0,2,...,0,1,1,0,3,1,1,0,0,1,4,2,1,2,0,0,0,0,2,1,1,0,0,2,1,0,0,0,1,0,1,2,1,0,1,0,1,1,4,0
1017,HOUSEHOLD_1_462_CA_1_validation,HOUSEHOLD_1_462,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,4,0,4,1,2,0,3,3,1,0,1,1,0,0,0,1,1,0,1,2,0,0,0,0,0,0,0,1,0,0,5,0,0,0,1,0,0,1
1018,HOUSEHOLD_1_463_CA_1_validation,HOUSEHOLD_1_463,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1019,HOUSEHOLD_1_464_CA_1_validation,HOUSEHOLD_1_464,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,1,0,2,0,0,0,0,7,0,0,2,0,6,0,0,0,0,1,0,0,1,0,0,0,2,1,1,1,0,2,1,0,1,...,1,0,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,2,0,0,0,0,0,0,1,0,0,2,1,0,0,0,0,1,0,1
1020,HOUSEHOLD_1_465_CA_1_validation,HOUSEHOLD_1_465,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,13,8,7,12,20,9,5,7,9,5,13,16,8,12,15,9,17,14,22,82,2,5,16,7,4,19,11,19,7,9,24,21,15,10,9,10,13,16,37,24


In [0]:
sales_train_df = sales_train_df.drop(columns=["item_id", "dept_id", "cat_id", "store_id", "state_id"])

In [0]:
sales_df = pd.melt(sales_train_df, id_vars=["id"], var_name="d")

In [21]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1943608 entries, 0 to 1943607
Data columns (total 3 columns):
id       object
d        object
value    int64
dtypes: int64(1), object(2)
memory usage: 44.5+ MB


In [22]:
sales_df.head()

,id,d,value
0,HOUSEHOLD_1_461_CA_1_validation,d_1,1
1,HOUSEHOLD_1_462_CA_1_validation,d_1,0
2,HOUSEHOLD_1_463_CA_1_validation,d_1,0
3,HOUSEHOLD_1_464_CA_1_validation,d_1,0
4,HOUSEHOLD_1_465_CA_1_validation,d_1,0


In [0]:
joined_df = sales_df.merge(date_map_df, on="d", how="left")

In [24]:
joined_df.head()

,id,d,value,ds,snap_CA,snap_TX,snap_WI,wm_yr_wk
0,HOUSEHOLD_1_461_CA_1_validation,d_1,1,2011-01-29,0,0,0,11101
1,HOUSEHOLD_1_462_CA_1_validation,d_1,0,2011-01-29,0,0,0,11101
2,HOUSEHOLD_1_463_CA_1_validation,d_1,0,2011-01-29,0,0,0,11101
3,HOUSEHOLD_1_464_CA_1_validation,d_1,0,2011-01-29,0,0,0,11101
4,HOUSEHOLD_1_465_CA_1_validation,d_1,0,2011-01-29,0,0,0,11101


In [0]:
final_joined_df = joined_df.merge(sell_prices_df, on=["id", "wm_yr_wk"], how="left")

In [26]:
final_joined_df.head()

,id,d,value,ds,snap_CA,snap_TX,snap_WI,wm_yr_wk,sell_price
0,HOUSEHOLD_1_461_CA_1_validation,d_1,1,2011-01-29,0,0,0,11101,0.97
1,HOUSEHOLD_1_462_CA_1_validation,d_1,0,2011-01-29,0,0,0,11101,NaN
2,HOUSEHOLD_1_463_CA_1_validation,d_1,0,2011-01-29,0,0,0,11101,NaN
3,HOUSEHOLD_1_464_CA_1_validation,d_1,0,2011-01-29,0,0,0,11101,8.87
4,HOUSEHOLD_1_465_CA_1_validation,d_1,0,2011-01-29,0,0,0,11101,NaN


In [0]:
proph_df = final_joined_df.drop(columns=["d"])

In [0]:
proph_df = proph_df.rename(columns={"value": "y"})

In [29]:
proph_df.ds.max()

'2016-04-24'

In [30]:
proph_df.ds.min()

'2011-01-29'

In [0]:
header = ["id"]
for i in range(1, 29):
  header.append("F" + str(i))

In [0]:
proph_op_df = pd.DataFrame(columns=header)

In [33]:
proph_op_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 29 columns):
id     0 non-null object
F1     0 non-null object
F2     0 non-null object
F3     0 non-null object
F4     0 non-null object
F5     0 non-null object
F6     0 non-null object
F7     0 non-null object
F8     0 non-null object
F9     0 non-null object
F10    0 non-null object
F11    0 non-null object
F12    0 non-null object
F13    0 non-null object
F14    0 non-null object
F15    0 non-null object
F16    0 non-null object
F17    0 non-null object
F18    0 non-null object
F19    0 non-null object
F20    0 non-null object
F21    0 non-null object
F22    0 non-null object
F23    0 non-null object
F24    0 non-null object
F25    0 non-null object
F26    0 non-null object
F27    0 non-null object
F28    0 non-null object
dtypes: object(29)
memory usage: 0.0+ bytes


In [0]:
grouped = proph_df.groupby('id')
i = 0
for g in grouped.groups:
    group = grouped.get_group(g)
    future_val = pd.DataFrame(future_dates, columns=["ds"])
    future_val = future_val.merge(date_map_df[1913:1941], on="ds")
    future_val['id'] = g
    future_val['y'] = 0
    future_val = future_val.merge(sell_prices_df, on=["id", "wm_yr_wk"], how="left")
    group = group.fillna(group.mean())
    future = group.append(future_val)
    m = Prophet()
    m.add_regressor("snap_CA")
    m.add_regressor("snap_TX")
    m.add_regressor("snap_WI")
    m.add_regressor("sell_price")
    m.fit(group)
    future = future.drop(columns=["d", "id", "y", "wm_yr_wk"])
    forecast = m.predict(future)
    res = forecast.tail(28)
    out = [g]
    for val in res["yhat"]:
        out.append(val)
    print(out)
    proph_op_df.loc[i] = out
    i += 1
    print(proph_op_df.tail())

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:7138: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


['FOODS_1_001_CA_1_validation', 0.6660789983355044, 0.6131423960614486, 0.7004677690130594, 0.6309952715475349, 0.9104260989705524, 0.9914594308754177, 0.9513484207688503, 0.793665853393517, 0.6906434103695368, 0.7863974751672552, 0.710553679895195, 0.9895114998812826, 1.023387131631838, 1.0773443910370042, 0.7656850607009036, 0.7187478060989102, 0.7421031851233928, 0.6691021918297312, 0.8982209902346775, 1.0723493490361498, 0.9716223382188477, 0.716352206346119, 0.6599739020904415, 0.7431539569534663, 0.6690850352168074, 0.9437282715419115, 1.0200400640522296, 0.973383383356814]
                            id        F1        F2  ...       F26      F27       F28
0  FOODS_1_001_CA_1_validation  0.666079  0.613142  ...  0.943728  1.02004  0.973383

[1 rows x 29 columns]


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:7138: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


['FOODS_1_002_CA_1_validation', 0.6644671656233495, 0.5560354299852397, 0.6432940880174857, 0.6703698428138427, 0.6624111586625352, 0.7272951243669388, 0.6818521165605468, 0.7988389336053404, 0.616031030939838, 0.6715388307404906, 0.6959156818707128, 0.670294459997595, 0.644602083609957, 0.7765695684327356, 0.6879719314928849, 0.5344336456012706, 0.6113510544618732, 0.610517603107626, 0.5036906213381191, 0.6771978013578261, 0.6165443565589882, 0.5715601260216593, 0.4357157400005437, 0.4991955170582742, 0.5066923726975199, 0.4838287661490471, 0.5388505042433182, 0.5515012320177428]
                            id        F1  ...       F27       F28
0  FOODS_1_001_CA_1_validation  0.666079  ...  1.020040  0.973383
1  FOODS_1_002_CA_1_validation  0.664467  ...  0.538851  0.551501

[2 rows x 29 columns]


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

['FOODS_1_027_CA_1_validation', 0.7826592948048547, 0.38645471290820177, 0.5214486636703756, 0.604290666736514, 0.7528156763687387, 0.9601787803757975, 0.44490685168076866, 0.11648045389977121, -0.3156875148071732, -0.02239863662424374, -0.016630090589090152, 0.18333360220693423, 0.5054464126413281, 0.17645427667095692, -0.157372566142508, -0.3361590679035069, -0.16435249304944782, 0.03074268491599197, 0.3538831497432997, 0.6878010587765371, 0.34590631964230867, 0.2502818119812827, -0.0041421139962936615, 0.27459795688712413, 0.5004604355691895, 0.7886230871124921, 1.1296761548628085, 0.8572116756466216]
                             id        F1  ...       F27       F28
21  FOODS_1_023_CA_1_validation -0.654995  ... -0.403976 -0.653530
22  FOODS_1_024_CA_1_validation  0.321010  ...  0.532998  0.228468
23  FOODS_1_025_CA_1_validation  0.642457  ...  0.653881  0.767894
24  FOODS_1_026_CA_1_validation  0.408547  ...  0.292048  0.180183
25  FOODS_1_027_CA_1_validation  0.782659  ...  1.129

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


['FOODS_1_028_CA_1_validation', 0.6683497516675316, 0.5367820846550349, 0.6089403207232695, 0.6828201172221398, 0.8358089719296822, 0.8499454656919923, 0.8572730233383136, 0.8450538527017123, 0.739353565712356, 0.6459775671883357, 0.8377998155676519, 0.9721274487783298, 0.885072307185361, 0.8515480826458471, 0.853825817296423, 0.5516210420860147, 0.7306541915710731, 0.7867930446204859, 0.8420194004147801, 0.883641959666057, 0.8822940784475941, 0.7025766435375915, 0.552556162985657, 0.6118408350030551, 0.6779030567459661, 0.8274233483451325, 0.8415802780041071, 0.7885822814979845]
                             id        F1  ...       F27       F28
22  FOODS_1_024_CA_1_validation  0.321010  ...  0.532998  0.228468
23  FOODS_1_025_CA_1_validation  0.642457  ...  0.653881  0.767894
24  FOODS_1_026_CA_1_validation  0.408547  ...  0.292048  0.180183
25  FOODS_1_027_CA_1_validation  0.782659  ...  1.129676  0.857212
26  FOODS_1_028_CA_1_validation  0.668350  ...  0.841580  0.788582

[5 rows x 

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


['FOODS_1_029_CA_1_validation', 0.41066223635999377, 0.3688297999661545, 0.3568530867976836, 0.3876530571849677, 0.3369254505311734, 0.33666446217681634, 0.3356693686499992, 0.3736932285775954, 0.3578011766294278, 0.28230771982428904, 0.3736319095165342, 0.3231286046710009, 0.29187182917424026, 0.26869541369692945, 0.33616404637037806, 0.23455592772916703, 0.28879795620118537, 0.3254380725980981, 0.2501508122241528, 0.26294736890872106, 0.2744129885973823, 0.2607045122477492, 0.23018479777348055, 0.23173774943589565, 0.2780912664659003, 0.2447103560609212, 0.2633247346153542, 0.2543586061250931]
                             id        F1  ...       F27       F28
23  FOODS_1_025_CA_1_validation  0.642457  ...  0.653881  0.767894
24  FOODS_1_026_CA_1_validation  0.408547  ...  0.292048  0.180183
25  FOODS_1_027_CA_1_validation  0.782659  ...  1.129676  0.857212
26  FOODS_1_028_CA_1_validation  0.668350  ...  0.841580  0.788582
27  FOODS_1_029_CA_1_validation  0.410662  ...  0.263325  0.25

In [0]:
proph_op_df.to_csv(file_name + "_op.csv", index=False)